# Physics and Astrophysics

## Orbital Dynamics

### The Two-Body Problem


### Kepler's Laws of Planetary Motion


### Newton's Laws of Motion and Gravity


### Bound and Unbound Orbits

## Thermodynamics

### Laws of Thermodynamics

### Barometric Law and Hydrostatic Equilibrium

## Stellar Properties and Lifetimes

### Virial Theorem

### Luminosity

### Stellar Size

## Sizes and Densities of Massive Planets

## Nucleosynthesis

### Primordial Nucleosynthesis

### Stellar Nucleosynthesis

### Radioactive decay
